In [1]:
import numpy as np

import datetime as dt
import panel as pn
import concurrent.futures

pn.extension()
import panel as pn
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import json 
import pandas as pd 
# from pandas.json_normalize import json_normalize #package for flattening json in pandas df

import pathlib

import holoviews as hv
import matplotlib.pyplot as plt
import hvplot
import param
import hvplot.pandas
from bokeh.models import GlyphRenderer, LinearAxis, LinearScale, Range1d


pn.extension(sizing_mode='stretch_width')

pn.extension('perspective')
from cognite.client.utils import MIN_TIMESTAMP_MS, MAX_TIMESTAMP_MS

In [2]:
# Contact Project Administrator to get these
TENANT_ID = "3aa4a235-b6e2-48d5-9195-7fcf05b459b0" # AAD using @equinor.com
CLIENT_ID = "3d7a85da-05ab-4675-b5f9-0c4b96a1c84b" # CDF_ALL_USER_APPLICATION_FLOW
CDF_CLUSTER = "westeurope-1"
# -------------for DEV project-----------------
COGNITE_PROJECT = "equinor-dev"

# -------------for TEST project-----------------
# COGNITE_PROJECT = "equinor-test"


from cognite.client import CogniteClient
client = CogniteClient.default_oauth_interactive(
    project=COGNITE_PROJECT,
    tenant_id=TENANT_ID,
    client_id=CLIENT_ID,
    cdf_cluster=CDF_CLUSTER,
    client_name="my-cognite-python-test", # a name to identify your session
)
from cognite.client.data_classes import LabelFilter



In [3]:
#  
wellcom_dataset = "src:015:wellcom"
witsml_dataset = "src:014:witsmldata"
active_wellbore = "active:wellbore"

#list wellcom wellbores that are active in WITSML
active_wellbores = client.assets.list(data_set_external_ids = wellcom_dataset, labels=LabelFilter(contains_all=[active_wellbore]), limit=-1)
#Get sensor data 
witsml_db  = client.time_series.list(data_set_external_ids = witsml_dataset, limit=-1)

In [4]:
# Wellbores
pd_active_wellbores = active_wellbores.to_pandas(metadata_prefix=active_wellbores)

# Sensordata
_pd_witsml = witsml_db.to_pandas(metadata_prefix=witsml_db)
_pd_witsml.rename(columns ={'name': 'sensorName'} , inplace = True)
# Flatten Sensordata
_pd_witsml_flattened = pd.concat([_pd_witsml.drop(['metadata'], axis=1), pd.json_normalize(_pd_witsml['metadata'] )], axis=1)





#Limiting columns
_pd_witsml_flattened_lim = _pd_witsml_flattened[['id', 'uidWell', 'sensorName', 'name', 'uidWellbore',  'nameWellbore','external_id',  'is_string', 'asset_id', 'is_step',
       'description', 'security_categories', 'data_set_id', 'created_time',
       'last_updated_time', 'type', 'commonData_dTimCreation',
       'commonData_dTimLastChange', 'commonData_itemState',
       'commonData_sourceName', 'creationDate', 'direction',
       'endDateTimeIndex', 'indexCurve', 'indexType',  'nameWell',
       'nullValue', 'objectGrowing', 'serviceCompany',
       'startDateTimeIndex',  'uidLog',
       'unavailable_in_source', 'witsml-server-ref', 'source',
       'curveDescription', 'maxDateTimeIndex', 'minDateTimeIndex', 'mnemAlias',
       'mnemonic', 'typeLogData', 'uid']]




In [5]:
sensor_ids = [
    5475659810979685,
    5772085538835957,
    3050562702196401
]

def retrieve_sensor_data(sensor_id , start , end):
    return client.time_series.data.retrieve(id= sensor_id, start = start, end = end)

retrieve_sensor_data(5475659810979685 , MIN_TIMESTAMP_MS, MAX_TIMESTAMP_MS)

,dw:witsml:log.49a36410-e7fb-4864-b3c7-c1eba937dcd9.0ac9cfa3-b6a0-4af3-85e4-3180f68f0c11.0ac9cfa3-b6a0-4af3-85e4-3180f68f0c11.blockcomp||v1
2022-11-24 05:26:56,0.328597
2022-11-24 05:26:57,0.328597
2022-11-24 05:26:58,0.328597
2022-11-24 05:26:59,0.328597
2022-11-24 05:27:00,0.328597
...,...
2023-01-11 17:29:43,13.698301
2023-01-11 17:29:44,13.698301
2023-01-11 17:29:45,13.698301
2023-01-11 17:29:46,13.698301


In [7]:

#Get name and id of wellbore
pd_wellbores = _pd_witsml_flattened[['nameWellbore', 'uidWellbore']]
#Get rid of duplicates
pd_wellbores = pd_wellbores.drop_duplicates(subset=['uidWellbore'])
#Create dictionary 
wellbore_map = {row['nameWellbore']: row['uidWellbore'] for index, row in pd_wellbores.iterrows()}

#Get name and id of sensor
pd_sensors = _pd_witsml_flattened[['sensorName', 'id', 'external_id']]
#Get rid of duplicates
pd_sensors = pd_sensors.drop_duplicates(subset=['id'])
 

In [17]:



# Defining the parameterized class 
class CustomDashboard(param.Parameterized):
    
    #Parameter for wellbore selector 
    wellbore = param.ObjectSelector(default=pd_wellbores['nameWellbore'].iloc[0], objects=list( sorted(pd_wellbores['nameWellbore'])    ))
    #sensor =   sensor = param.ObjectSelector(default= _pd_witsml_flattened[_pd_witsml_flattened['nameWellbore'] == pd_wellbores['nameWellbore'].iloc[0] ]['sensorName'].iloc[0]   , objects=list( _pd_witsml_flattened[_pd_witsml_flattened['nameWellbore'] == pd_wellbores['nameWellbore'].iloc[0] ]['sensorName']) ) 
    sensor = param.Selector( objects=list() ) 

    
    @param.depends('wellbore', watch=True)
    def update_sensors(self):
        wellbore_id = wellbore_map[self.wellbore]
        filtered_sensors = _pd_witsml_flattened[_pd_witsml_flattened['uidWellbore'] == wellbore_id]
        sensor_names = list(sorted(filtered_sensors['sensorName']))
        self.param['sensor'].objects = sensor_names
        if sensor_names:
                self.sensor = sensor_names[0]
        else:
            self.sensor= None
        

    @param.depends('wellbore', 'sensor')
    def get_sensor_id(self):
        wellbore_id= wellbore_map[self.wellbore]
        sensor_data = _pd_witsml_flattened[(_pd_witsml_flattened['uidWellbore'] == wellbore_id) & (_pd_witsml_flattened['sensorName'] == self.sensor)]
        if not sensor_data.empty:
            return sensor_data['id'].iloc[0]
        return None
    
    @param.depends('wellbore', 'sensor')
    def view(self):
        accessible_value = self.get_sensor_id()
        if accessible_value:
                return pn.pane.Str(f'Selected Sensor ID: {accessible_value}')
        return pn.pane.Str('No sensor selected')
    
    

    @param.depends('sensor')
    def plot(self):
        if self.sensor:
            #Getting sensor id
            sensor_id = int(self.get_sensor_id())
            #Fetching sensor data
            sensor_data = client.time_series.data.retrieve(id=sensor_id,start=MIN_TIMESTAMP_MS,  end=MAX_TIMESTAMP_MS + 1)
            #Converting sensor data to pandas
            pd_sensor_data = sensor_data.to_pandas()
            
            # Reindexing the sensor data
            pd_sensor_data['Value'] = pd_sensor_data[pd_sensor_data.columns]
            pd_sensor_data['Date'] = pd.to_datetime(pd_sensor_data.index)
            pd_sensor_data['Date_Seconds'] = pd_sensor_data['Date'].apply(lambda x: x.timestamp())

            #Adding a more consistant series of time stamps
            #TODO remember to interpolate date column
            
            start_date = pd_sensor_data['Date'].min()
            end_date = pd_sensor_data['Date'].max()
            #Finding the smallest interval to define the frequency
            time_diff = pd_sensor_data['Date'].diff().min()
            new_timestamps = pd.date_range(start=start_date, end=end_date, freq= time_diff)

            #Create a new DataFrame with the new timestamps
            new_data = pd.DataFrame(new_timestamps, columns=['New_Timestamps'])

            #Merge the new data with the original data using an outer join
            pd_sensor_data = pd.merge(pd_sensor_data, new_data, how='outer', left_on='Date', right_on='New_Timestamps')

            #Joining the new timestamps 
            pd_sensor_data.set_index('New_Timestamps')

            pd_sensor_data

            # Count the number of rows with null in the 'Date' field
            null_dates = pd_sensor_data[pd_sensor_data['Date'].isnull()]

            # Print the result
            
            


        
            if not pd_sensor_data.empty:
                plot = pd_sensor_data.hvplot.scatter(x='Date', y='New_Timestamps', autorange="y", title=f'THIS IS THE DEV VERSION {self.sensor}').opts(invert_yaxis=True, width = 500, height = 1000)
                return plot
        
    
    #pn.pane.Str('No data available for the selected sensor')
    
    #Run update sensor method to populate sensor dropdown
    def __init__(self):
        self.update_sensors()


    def panel(self):
        return pn.Column(
            pn.Param(self.param, widgets={'wellbore': pn.widgets.Select, 'sensor': pn.widgets.Select}),
            self.view, self.plot
        )

# Instantiate the dashboard
dashboard = CustomDashboard()

dashboard.plot()

# Trigger initial sensor update
#dashboard.update_sensors()

# Display the dashboard
dashboard.panel().show()





C:\Users\mats.vaage\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\panel\param.py:624: ParamFutureWarning: (Un)registering a watcher on a partially initialized Parameterized instance is deprecated and will raise an error in a future version. Ensure you have called super().__init__(**) in the Parameterized instance constructor before trying to set up a watcher.
  watchers.append(self.object.param.watch(link, p_name, 'constant'))
C:\Users\mats.vaage\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\panel\param.py:625: ParamFutureWarning: (Un)registering a watcher on a partially initialized Parameterized instance is deprecated and will raise an error in a future version. Ensure you have called super().__init__(**) in the Parameterized instance constructor before trying to set up a watcher.
  watchers.append(self.object.param.watch(link, p_n

Launching server at http://localhost:56719


# Concurrency test

In [ ]:
# Wellbores
pd_active_wellbores = active_wellbores.to_pandas(metadata_prefix=active_wellbores)

# Sensordata
_pd_witsml = witsml_db.to_pandas(metadata_prefix=witsml_db)
_pd_witsml.rename(columns ={'name': 'sensorName'} , inplace = True)
# Flatten Sensordata

# 5475659810979685
# 5772085538835957
# 3050562702196401
# 1293082170891378
# make datasets
sensor_ids = [
    5475659810979685,
    5772085538835957,
    3050562702196401
]

def retrieve_sensor_data(sensor_id , start , end):
    return client.time_series.data.retrieve(id= sensor_id, start = start, end = end)


def retrieve_prep_sensor_data(sensor_ids) : 
    # Use ThreadPoolExecutor to parallelize the requests
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Submit each sensor data retrieval task to the executor
        future_to_sensor_id = {executor.submit(retrieve_sensor_data, sensor_id, MIN_TIMESTAMP_MS, MAX_TIMESTAMP_MS): sensor_id for sensor_id in sensor_ids}
        
        # Collect the results as they complete
        sensor_data_results = {}
        for future in concurrent.futures.as_completed(future_to_sensor_id):
            sensor_id = future_to_sensor_id[future]
            try:
                data = future.result()
                data = data.to_pandas()
                data['Value'] = data[data.columns]
                data['Date'] = pd.to_datetime(data.index)
                data['Date_Seconds'] = data['Date'].apply(lambda x: x.timestamp())
                sensor_data_results[sensor_id] = data

            except Exception as exc:
                print(f'Sensor {sensor_id} generated an exception: {exc}')

    return sensor_data_results

# retrieve_prep_sensor_data(sensor_ids)

sensor_data =  retrieve_prep_sensor_data(sensor_ids)
combined_data = pd.DataFrame()
for sensor_id, data in sensor_data.items():
    data['sensor_id'] = sensor_id
    combined_data = pd.concat([combined_data, data[['Date', 'Value', 'sensor_id']]])

# Plotting the combined data using HvPlot
combined_plot = combined_data.hvplot.line(x='Date', y='Value', by='sensor_id', title='Combined Sensor Data', legend='top')


# Show the plot
combined_plot
pn.Column(combined_plot).show()


_pd_witsml.drop(['metadata'], axis=1), pd.json_normalize(_pd_witsml['metadata'] )], axis=1)

In [19]:
def rectangle_test():


    rectangles = pd.DataFrame({
    'x': [0, 0, 0],
    'ystartRect': [pd.Timestamp('2022-11-26 07:42:42'), pd.Timestamp('2022-01-27 05:27:00'), pd.Timestamp('2022-01-28 05:29:00')],
    'yendRect': [pd.Timestamp('2022-11-29 05:27:00'), pd.Timestamp('2022-01-28 05:27:00'), pd.Timestamp('2022-01-29 05:27:00')],
    'widthRect': [20000000000000.00, 20000000000000.00, 20000000000000.00],
    'labelRect': ['Rectangle 1 - U', 'Rectangle 2 - D', 'Rectangle 3 - U'] })
        
         
    for d in rectangles:
                # set colors of rectanlges
        if '- D' in d['label']:
            fill_color = 'red'
        elif '- U' in d['label']:
            fill_color = 'green'
        else:
            fill_color = 'orange'

        x0, y0 = d['x'], d['ystart']
        x1, y1 = d['x'] + d['xend'], d['yend']
        vertices = [(x0, y0), (x1, y0), (x1, y1), (x0, y1)]
        print(vertices)

        
rectangle_test()

TypeError: string indices must be integers, not 'str'